# ML module 1

In [100]:
'''
1) Загрузка данных
2) Исследование данных
3) Подготовка данных к обучению модели
4) Моделирование
5) Предсказание тестовой выборки

'''

'\n1) Загрузка данных\n2) Исследование данных\n3) Подготовка данных к обучению модели\n4) Моделирование\n5) Предсказание тестовой выборки\n\n'

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing

## 1) Загрузка данных

In [7]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

display(train.head())
display(test.head())

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


## 2) EDA

### Типы данных

In [10]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

### Количество уникальных значений в разных полях

In [15]:
train.nunique()

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64

### Проверка пропусков

In [18]:
train.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [22]:
'''
Age, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - численные значения
PassengerId, Cabin, Name - строки
HomePlanet, CryoSleep, Destination, VIP - категориальные значения
CryoSleep, VIP - булевые значения
Transported - целевое значение
'''

'\nAge, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - численные значения\nPassengerId, Cabin, Name - строки\nHomePlanet, CryoSleep, Destination, VIP - категориальные значения\nCryoSleep, VIP - булевые значения\nTransported - целевое значение\n'

## 3) Подготовка данных

### Заполнение пропусков

In [30]:
# Список колонок, которые заполню null
exclude_cols = ['PassengerId', 'Cabin', 'Name']

# Заполнение пропусков
for col in train.columns:
    if col in exclude_cols: # Пропускаем эти колонки
        continue

    if train[col].dtype == 'object':  # Категориальные переменные
        train[col] = train[col].fillna(train[col].mode()[0]).infer_objects(copy=False) # Cамое часто встречающееся
    else:  # Числовые переменные
        train[col] = train[col].fillna(train[col].median()) # Среднее значение после сортировки

train['Cabin'] = train['Cabin'].fillna('None')
train['Name'] = train['Name'].fillna('None')

train.isna().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

### Добавляем колонки

In [33]:
# Добавляю колонку трат 
cost_col = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
train['Cost'] = train[cost_col].sum(axis=1)
train['haveCost'] = train['Cost'] > 0
train['LogCost'] = np.log1p(train['Cost'])
train[['Cost', 'LogCost', 'haveCost']].head()

,Cost,LogCost,haveCost
0,0.0,0.000000,False
1,736.0,6.602588,True
2,10383.0,9.248021,True
3,5176.0,8.551981,True
4,1091.0,6.995766,True


In [39]:
# Разбиваю кабины
cabin = train['Cabin'].str.split("/", expand=True)
cabin.columns = ['Deck', 'Num', 'Side']
train['Deck'] = cabin['Deck']
train['Num'] = cabin['Num'].fillna(-1)
train['Side'] = cabin['Side'].fillna('None')

train[['Cabin', 'Deck', 'Num', 'Side']].head()

,Cabin,Deck,Num,Side
0,B/0/P,B,0,P
1,F/0/S,F,0,S
2,A/0/S,A,0,S
3,A/0/S,A,0,S
4,F/1/S,F,1,S


In [43]:
# Проверка на соло путешественника
train['GroupId'] = train['PassengerId'].apply(lambda x: x.split("_")[0]).astype(int)
train['SoloTraveler'] = train['GroupId'].map(train['GroupId'].value_counts()) == 1

train[['GroupId', 'SoloTraveler']].head()

,GroupId,SoloTraveler
0,1,True
1,2,True
2,3,False
3,3,False
4,4,True


### Обрабатываем по разному разные типы данных

In [48]:
# Делим столбцы, с которыми будем работать по категориям

cat_cols = ['HomePlanet', 'Destination', 'Deck', 'Side']
num_cols = ['Age', 'LogCost', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa',
            'VRDeck', 'Num', 'SoloTraveler', 'CryoSleep', 'haveCost', 'VIP']

### Категориальные значения

In [51]:
from sklearn.preprocessing import OneHotEncoder

In [53]:
train[cat_cols]

,HomePlanet,Destination,Deck,Side
0,Europa,TRAPPIST-1e,B,P
1,Earth,TRAPPIST-1e,F,S
2,Europa,TRAPPIST-1e,A,S
3,Europa,TRAPPIST-1e,A,S
4,Earth,TRAPPIST-1e,F,S
...,...,...,...,...
8688,Europa,55 Cancri e,A,P
8689,Earth,PSO J318.5-22,G,S
8690,Earth,TRAPPIST-1e,G,S
8691,Europa,55 Cancri e,E,S


In [55]:
train[cat_cols[0]].value_counts()

HomePlanet
Earth     4803
Europa    2131
Mars      1759
Name: count, dtype: int64

In [57]:
ohe = OneHotEncoder(sparse_output=False)

ohe.fit(train[cat_cols])

OneHotEncoder(sparse_output=False)

In [59]:
display(ohe.categories_)

cat_col_array = np.concatenate(ohe.categories_)
cat_col_array

[array(['Earth', 'Europa', 'Mars'], dtype=object),
 array(['55 Cancri e', 'PSO J318.5-22', 'TRAPPIST-1e'], dtype=object),
 array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'None', 'T'], dtype=object),
 array(['None', 'P', 'S'], dtype=object)]

array(['Earth', 'Europa', 'Mars', '55 Cancri e', 'PSO J318.5-22',
       'TRAPPIST-1e', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'None', 'T',
       'None', 'P', 'S'], dtype=object)

In [61]:
cat_cols_transform = ohe.transform(train[cat_cols])
cat_cols_transform

array([[0., 1., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.]])

In [63]:
display(train[cat_cols].iloc[0])
display(cat_cols_transform[0])

HomePlanet          Europa
Destination    TRAPPIST-1e
Deck                     B
Side                     P
Name: 0, dtype: object

array([0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0.])

### Численнные значения

In [66]:
train[num_cols]

,Age,LogCost,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Num,SoloTraveler,CryoSleep,haveCost,VIP
0,39.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,True,False,False,False
1,24.0,6.602588,109.0,9.0,25.0,549.0,44.0,0,True,False,True,False
2,58.0,9.248021,43.0,3576.0,0.0,6715.0,49.0,0,False,False,True,True
3,33.0,8.551981,0.0,1283.0,371.0,3329.0,193.0,0,False,False,True,False
4,16.0,6.995766,303.0,70.0,151.0,565.0,2.0,1,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41.0,9.052165,0.0,6819.0,0.0,1643.0,74.0,98,True,False,True,True
8689,18.0,0.000000,0.0,0.0,0.0,0.0,0.0,1499,True,True,False,False
8690,26.0,7.535830,0.0,0.0,1872.0,1.0,0.0,1500,True,False,True,False
8691,32.0,8.442039,0.0,1049.0,0.0,353.0,3235.0,608,False,False,True,False


In [68]:
train[num_cols].isna().sum()

Age             0
LogCost         0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Num             0
SoloTraveler    0
CryoSleep       0
haveCost        0
VIP             0
dtype: int64

## 4) Моделирование

In [71]:
# Подготавливаю обучающие массивы
numeric = train[num_cols].values
display('Численные переменные',
        numeric)

display('Закодированные категориальные переменные',
        cat_cols_transform)

X = np.concatenate([numeric,
                    cat_cols_transform], axis=1)

X_columns = num_cols + list(cat_col_array)
display(X_columns)

y = train['Transported'].values

'Численные переменные'

array([[39.0, 0.0, 0.0, ..., False, False, False],
       [24.0, 6.602587892189336, 109.0, ..., False, True, False],
       [58.0, 9.248021438943871, 43.0, ..., False, True, True],
       ...,
       [26.0, 7.535830462798367, 0.0, ..., False, True, False],
       [32.0, 8.442038517815478, 0.0, ..., False, True, False],
       [44.0, 8.481980435660493, 126.0, ..., False, True, False]],
      dtype=object)

'Закодированные категориальные переменные'

array([[0., 1., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.]])

['Age',
 'LogCost',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'Num',
 'SoloTraveler',
 'CryoSleep',
 'haveCost',
 'VIP',
 'Earth',
 'Europa',
 'Mars',
 '55 Cancri e',
 'PSO J318.5-22',
 'TRAPPIST-1e',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'None',
 'T',
 'None',
 'P',
 'S']

### Делим выборки

In [74]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=42)

display(X_train.shape,
        X_valid.shape,
        y_train.shape,
        y_valid.shape)

(6954, 30)

(1739, 30)

(6954,)

(1739,)

### Optuna + CatBoost

In [89]:
import optuna
import pickle
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score,classification_report
from sklearn.datasets import load_breast_cancer

def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 1000, 3000, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 15.0),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.01, 10.0),
        "random_strength": trial.suggest_float("random_strength", 0.01, 15.0),
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 2, 20),
        "leaf_estimation_method": trial.suggest_categorical("leaf_estimation_method", ["Newton", "Gradient"]),
        "depth": trial.suggest_int("depth", 4, 12),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "loss_function": "Logloss"
    }

    model = CatBoostClassifier(**params, cat_features=None, verbose=False)
    model.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=False)

    preds = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, preds)

    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print('Лучший результат', study.best_trial.params)

[I 2025-03-25 23:01:30,094] A new study created in memory with name: no-name-c89ed66a-eda7-44b8-8e07-64405987804b
[I 2025-03-25 23:02:50,482] Trial 0 finished with value: 0.8027602070155262 and parameters: {'iterations': 3000, 'learning_rate': 0.15368326452579145, 'l2_leaf_reg': 7.555616319793951, 'bagging_temperature': 9.64891007040432, 'random_strength': 10.166547240659495, 'one_hot_max_size': 8, 'leaf_estimation_method': 'Gradient', 'depth': 11, 'border_count': 157}. Best is trial 0 with value: 0.8027602070155262.
[I 2025-03-25 23:02:54,313] Trial 1 finished with value: 0.7981598619896493 and parameters: {'iterations': 1900, 'learning_rate': 0.005971276349867686, 'l2_leaf_reg': 1.8884546104006612, 'bagging_temperature': 3.5546703695554767, 'random_strength': 7.265674331792638, 'one_hot_max_size': 8, 'leaf_estimation_method': 'Newton', 'depth': 6, 'border_count': 146}. Best is trial 0 with value: 0.8027602070155262.
[I 2025-03-25 23:03:33,991] Trial 2 finished with value: 0.801035077

Лучший результат {'iterations': 1500, 'learning_rate': 0.13078343668146164, 'l2_leaf_reg': 10.24651815070874, 'bagging_temperature': 2.648267534551282, 'random_strength': 12.207895480308732, 'one_hot_max_size': 14, 'leaf_estimation_method': 'Gradient', 'depth': 6, 'border_count': 88}


### Смотрим результат

In [92]:
best_trial = study.best_trial.params
best_catboost = CatBoostClassifier(**best_trial, verbose=0)
best_catboost.fit(X_train, y_train)

y_pred = best_catboost.predict(X_valid)

In [94]:
sum(y_pred == y_valid)/ y_pred.shape

array([0.80448534])

In [96]:
print(accuracy_score(y_valid, y_pred))
confusion_matrix(y_valid, y_pred)

0.80448533640023


array([[691, 170],
       [170, 708]])

In [98]:
print (classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

       False       0.80      0.80      0.80       861
        True       0.81      0.81      0.81       878

    accuracy                           0.80      1739
   macro avg       0.80      0.80      0.80      1739
weighted avg       0.80      0.80      0.80      1739



## 5) Предсказание тестовой выборки

In [106]:
display()
test.isna().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [110]:
# Пропуски, колонки, категории
exclude_cols_test = ['PassengerId', 'Cabin', 'Name']

for col_test in test.columns:
    if col_test in exclude_cols_test:
        continue

    if test[col_test].dtype == 'object':
        test[col_test] = test[col_test].fillna(test[col_test].mode()[0]).infer_objects(copy=False)
    else:
        test[col_test] = test[col_test].fillna(test[col_test].median())

test['Cabin'] = test['Cabin'].fillna('None')
test['Name'] = test['Name'].fillna('None')

cost_col_test = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
test['Cost'] = test[cost_col_test].sum(axis=1)
test['haveCost'] = test['Cost'] > 0
test['LogCost'] = np.log1p(test['Cost'])

cabin_test = test['Cabin'].str.split("/", expand=True)
cabin_test.columns = ['Deck', 'Num', 'Side']
test['Deck'] = cabin_test['Deck']
test['Num'] = cabin_test['Num'].fillna(-1)
test['Side'] = cabin_test['Side'].fillna('None')

test['GroupId'] = test['PassengerId'].apply(lambda x: x.split("_")[0]).astype(int)
test['SoloTraveler'] = test['GroupId'].map(train['GroupId'].value_counts()) == 1

test.isna().sum()

/var/folders/lr/6trkdqrj6d1fq_j3stnp2tjw0000gn/T/ipykernel_84456/3566415455.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test[col_test] = test[col_test].fillna(test[col_test].mode()[0]).infer_objects(copy=False)


PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Cost            0
haveCost        0
LogCost         0
Deck            0
Num             0
Side            0
GroupId         0
SoloTraveler    0
dtype: int64

In [112]:
cat_cols_transform_test = ohe.transform(test[cat_cols])

numeric_test = test[num_cols].values

X_test = np.concatenate([numeric_test,
                        cat_cols_transform_test], axis=1)

predictions = best_catboost.predict(X_test)
predictions

array([ True, False,  True, ...,  True,  True, False])

### Обучаю итоговую модель и записываю результат

In [117]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Transported': predictions})
output.to_csv('submission.csv', index=False)

result = pd.read_csv('submission.csv')
result.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
